In [1]:
# OPENCV
import cv2 as cv
import numpy as np
from PIL import ImageGrab
from functools import partial
import pyautogui

ImageGrab.grab = partial(ImageGrab.grab, all_screens=True)

In [2]:
def hsvColorSlider():
    cv.namedWindow('HSV Color Slider')
    cv.createTrackbar('Hue Min', 'HSV Color Slider', 0, 179, nothing)
    cv.createTrackbar('Hue Max', 'HSV Color Slider', 104, 179, nothing)
    cv.createTrackbar('Sat Min', 'HSV Color Slider', 0, 255, nothing)
    cv.createTrackbar('Sat Max', 'HSV Color Slider', 134, 255, nothing)
    cv.createTrackbar('Val Min', 'HSV Color Slider', 108, 255, nothing)
    cv.createTrackbar('Val Max', 'HSV Color Slider', 255, 255, nothing)

def nothing(x):
    pass

def getHSV():
    hue_min = cv.getTrackbarPos('Hue Min', 'HSV Color Slider')
    hue_max = cv.getTrackbarPos('Hue Max', 'HSV Color Slider')
    sat_min = cv.getTrackbarPos('Sat Min', 'HSV Color Slider')
    sat_max = cv.getTrackbarPos('Sat Max', 'HSV Color Slider')
    val_min = cv.getTrackbarPos('Val Min', 'HSV Color Slider')
    val_max = cv.getTrackbarPos('Val Max', 'HSV Color Slider')
    return hue_min, hue_max, sat_min, sat_max, val_min, val_max

# hue_min = 0
# hue_max = 179
# sat_min = 0
# sat_max = 255
# val_min = 124
# val_max = 255

In [3]:
def convert_to_binary(image):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    # remove background
    (hmin, hmax, smin, smax, vmin, vmax) = getHSV()
    lower = np.array([hmin, smin, vmin])
    upper = np.array([hmax, smax, vmax])
    mask = cv.inRange(hsv, lower, upper)

    krn = cv.getStructuringElement(cv.MORPH_RECT, (50, 30))
    dlt = cv.dilate(mask, krn, iterations=5)
    res = 255 - cv.bitwise_and(dlt, mask)
    # # Apply threshold
    # _, res = cv.threshold(hsv, 0, 255, cv.THRESH_OTSU)
    return res

In [4]:
# detect checkers board
def detect_checkers_board(img):
    # convert to gray scale
    mask = convert_to_binary(img)
    # mask = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # find the chess board corners
    ret, corners = cv.findChessboardCorners(mask, (7, 7), None)
    # if found, draw corners
    if ret == True:
        # draw corners
        cv.drawChessboardCorners(img, (7, 7), corners, ret)
        return True, corners
    return False, None

In [5]:
# get the board corners
def get_board_corners(corners):
    # get the corners
    corners = corners.reshape(-1, 2)
    x_delta = abs(corners[0][0] - corners[1][0])
    y_delta = abs(corners[0][1] - corners[7][1])

    # get the top left corner
    top_left = corners[0][0] - x_delta, corners[0][1] - y_delta
    # get the top right corner
    top_right = corners[6][0] + x_delta, corners[6][1] - y_delta
    # get the bottom left corner
    bottom_left = corners[42][0] - x_delta, corners[42][1] + y_delta
    # get the bottom right corner
    bottom_right = corners[48][0] + x_delta, corners[48][1] + y_delta
    return top_left, top_right, bottom_left, bottom_right

In [6]:
def get_board_squares(squareCoords, corners):
    # get the corners
    corners = corners.reshape(-1, 2)
    x_delta = abs(corners[0][0] - corners[1][0])
    y_delta = abs(corners[0][1] - corners[7][1])
    
    home_coords = corners[0][0] - x_delta/2, corners[0][1] - y_delta/2

    x = home_coords[0] + ((x_delta) * squareCoords[1])
    y = home_coords[1] + ((y_delta) * squareCoords[0])
    coords = (x, y)

    return coords, x_delta, y_delta

In [7]:
def crop_board(img, corners):
    # get the corners
    top_left, top_right, bottom_left, bottom_right = get_board_corners(corners)
    # get the width of the board
    width = int(np.sqrt((top_right[0] - top_left[0])**2 + (top_right[1] - top_left[1])**2))
    # get the height of the board
    height = int(np.sqrt((top_left[0] - bottom_left[0])**2 + (top_left[1] - bottom_left[1])**2))
    # get the perspective transform matrix
    pts1 = np.float32([top_left, top_right, bottom_left, bottom_right])
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    matrix = cv.getPerspectiveTransform(pts1, pts2)
    # apply the perspective transform
    result = cv.warpPerspective(img, matrix, (width, height))
    return result

In [8]:
def detect_pieces(boardimg, corners):
    if corners is not None:
        # get the corners
        top_left, _, _, bottom_right = get_board_corners(corners)
        # detect white squares
        # convert to gray scale
        gray = cv.cvtColor(boardimg, cv.COLOR_BGR2GRAY)
        # apply threshold
        _, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
        # find contours
        contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        # get the squares
        # apply only the squares that are inside the board
        pieces = []
        for cnt in contours:
            x, y, w, h = cv.boundingRect(cnt)
            if bottom_right[0] > x > top_left[0] and bottom_right[1] > y > top_left[1]:
                if 25 < w < 50 and 25 < h < 50:
                    pieces.append((x, y, w, h))
    return pieces

In [9]:
def match_images(img, template):
    # convert to gray scale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    # get the template size
    w, h = template.shape[::-1]
    # match the template
    res = cv.matchTemplate(gray, template, cv.TM_CCOEFF_NORMED)
    # get the threshold
    threshold = 0.8
    loc = np.where(res >= threshold), w, h

    return loc

In [10]:
def getMove(prevBoard, newBoard, turn):
    convertTurn = {'b': (1, 3), 'w': (2, 4)}
    selected = None
    move = None
    pieceType = None
    for row in range(8):
        for col in range(8):
            if prevBoard[row][col] != 0 and newBoard[row][col] == 0 and prevBoard[row][col] in convertTurn[turn]:
                selected = (row, col)
                pieceType = prevBoard[row][col]
            if prevBoard[row][col] == 0 and newBoard[row][col] != 0:
                move = (row, col)
    
    if selected is not None and move is not None and pieceType is not None:
        if pieceType == 1 and move[0] == 7:
            pieceType = 3
        elif pieceType == 2 and move[0] == 0:
            pieceType = 4

    return selected, move, pieceType

In [11]:
import copy
def update_board_state(prevBoard, currBoard, turn):
    convertTurn = {'b': (1, 3), 'w': (2, 4)}
    otherPlayer = 'b' if turn == 'w' else 'w'
    updated_board = copy.deepcopy(prevBoard)
    capturedPiece = []

    for row in range(8):
        for col in range(8):
            if currBoard[row][col] == 1 and prevBoard[row][col] == 0:
                _, _, pieceType = getMove(prevBoard, currBoard, turn)
                print(pieceType)
                updated_board[row][col] = pieceType
            elif currBoard[row][col] == 0 and prevBoard[row][col] != 0:
                updated_board[row][col] = 0
                if prevBoard[row][col] in convertTurn[otherPlayer]:
                    capturedPiece.append((row, col))
    
    return updated_board, capturedPiece

In [12]:
def getBoardState(boardimg, corners):
    pieces = detect_pieces(boardimg, corners)
    board = np.zeros((8, 8))
    sorted_pieces = []

    for row in range(8):
        for col in range(8):
            square = (row, col)
            coords, x_delta, y_delta = get_board_squares(square, corners)
            for _, piece in enumerate(pieces):
                x, y, _, _ = piece
                if coords[0] - x_delta/2 < x < coords[0] + x_delta/2 and coords[1] - y_delta/2 < y < coords[1] + y_delta/2:
                    board[row][col] = 1
                    sorted_pieces.append((piece, square, coords))
                    break
    return board, sorted_pieces

In [13]:
def getMultipleCapture(prevBoard, newBoard, turn, captured):
    selected, move, _ = getMove(prevBoard, newBoard, turn)
    selectedList = [selected]
    moves = []
    
    deltaRow = move[0] - selected[0]
    deltaCol = move[1] - selected[1]
    if deltaRow < 0 or deltaCol < 0:
        captured = captured[::-1]
    
    tempBoard = copy.deepcopy(newBoard)
    for row, col in captured:
        rowDelta = row - selected[0]
        rowDelta = rowDelta // abs(rowDelta)
        colDelta = col - selected[1]
        colDelta = colDelta // abs(colDelta)
        selected = (row + rowDelta, col + colDelta)
        moves.append(selected)
    
    selectedList.extend(moves[:-1])

    return selectedList, moves

In [14]:
prevBoard = np.array([[0, 0, 0, 3, 0, 0, 0, 1],
                      [0, 0, 2, 0, 1, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 2, 0, 0, 0, 2, 0, 2],
                      [0, 0, 0, 0, 0, 0, 0, 0]])

testBoard = np.array([[0, 0, 0, 0, 0, 0, 0, 1],
                      [0, 0, 0, 0, 1, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 0, 0, 0, 1],
                      [0, 0, 0, 0, 0, 0, 1, 0]])

selected, move, pieceType = getMove(prevBoard, testBoard, 'b')
selected, move, pieceType

((0, 3), (7, 6), 3)

In [15]:
prevBoard = np.array([[0, 0, 0, 0, 0, 0, 0, 1],
                      [0, 0, 2, 0, 1, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 2, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 2, 0, 0, 0, 0, 0, 2],
                      [0, 0, 0, 0, 0, 0, 3, 0]])

testBoard = np.array([[0, 0, 0, 1, 0, 0, 0, 1],
                      [0, 0, 0, 0, 1, 0, 1, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 0, 0, 0, 1],
                      [0, 0, 0, 0, 0, 0, 0, 0]])

selected, move, pieceType = getMove(prevBoard, testBoard, 'b')
selected, move, pieceType

((7, 6), (0, 3), 3)

In [16]:
# selectedList, moves = getMultipleCapture(prevBoard, testBoard, 'b', capturedPieces)
# print(selectedList, moves)

NameError: name 'capturedPieces' is not defined

In [ ]:
updatedBoard, capturedPieces = update_board_state(prevBoard, testBoard, 'b')

print(capturedPieces)
for row in updatedBoard:
    print(row)

3
[(1, 2), (3, 2)]
[0 0 0 3 0 0 0 1]
[0 0 0 0 1 0 1 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 2 0 0 0 0 0 2]
[0 0 0 0 0 0 0 0]


In [ ]:
turn = 'w'
testResetBoard = copy.deepcopy(prevBoard)
# selected, move = (4, 5), (0, 1)
# selected, move = (0, 1), (4, 5)
print(selected, move)
print((selected[0] - move[0])//abs(selected[0] - move[0]), (selected[1] - move[1])//abs(selected[1] - move[1]))
pieceType = testResetBoard[selected[0]][selected[1]]
testResetBoard[selected[0]][selected[1]] = 0
testResetBoard[move[0]][move[1]] = pieceType
if abs(selected[0] - move[0]) >= 2:
    rowDelta = (selected[0] - move[0])//abs(selected[0] - move[0])
    colDelta = (selected[1] - move[1])//abs(selected[1] - move[1])
    testResetBoard[move[0] + rowDelta][move[1] + colDelta] = 0

for row in testResetBoard:
    print(row)

(1, 4) (7, 6)
-1 -1
[0 0 0 0 0 1 0 1]
[1 0 1 0 0 0 0 0]
[0 1 0 0 0 2 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 2 0 0]
[0 0 0 0 0 0 0 0]
[0 2 0 2 0 0 0 0]
[2 0 2 0 2 0 1 0]


In [ ]:
# only screen 1
screen = ImageGrab.grab(all_screens=True)
print(f"Resolution: {screen.size}, Mode: {screen.mode}, Format: {screen.format}")

# screen right : (1920, 0, 3840, 1080)
# screen left : (0, 0, 1920, 1080)

screen = screen.crop((1920, 0, 3840, 1080))
screen = np.array(screen)

Resolution: (3840, 1080), Mode: RGB, Format: None


In [17]:
def initGame(numGames):
    turn = 'b'
    # initialize the board
    prevBoard = np.array([[0, 1, 0, 1, 0, 1, 0, 1],
                          [1, 0, 1, 0, 1, 0, 1, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 1, 0, 1, 0, 1, 0, 1],
                          [1, 0, 1, 0, 1, 0, 1, 0]])
    
    bot = 'b' if numGames % 2 == 0 else 'w'
    our = 'w' if numGames % 2 == 0 else 'b'
    return turn, prevBoard, bot, our


In [18]:
import numpy as np
testBoard = np.array([[0, 1, 0, 1, 0, 1, 0, 1],
                    [1, 0, 1, 0, 1, 0, 1, 0],
                    [0, 3, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 2, 0, 2, 0, 2, 0, 2],
                    [2, 0, 2, 0, 2, 0, 2, 0]])

print("     0 1 2 3 4 5 6 7")
print("    -----------------")
for idx, row in enumerate(testBoard):
    print(f"{idx} | {row}")

print(np.where(testBoard == 3)[0][0], np.where(testBoard == 3)[1][0])

     0 1 2 3 4 5 6 7
    -----------------
0 | [0 1 0 1 0 1 0 1]
1 | [1 0 1 0 1 0 1 0]
2 | [0 3 0 0 0 0 0 0]
3 | [0 0 0 0 0 0 0 0]
4 | [0 0 0 0 0 0 0 0]
5 | [0 0 0 0 0 0 0 0]
6 | [0 2 0 2 0 2 0 2]
7 | [2 0 2 0 2 0 2 0]
2 1


In [19]:
flipBoard = np.flip(testBoard, 0)
flipBoard = np.flip(flipBoard, 1)
for row in flipBoard:
    print(row)
print(np.where(flipBoard == 3)[0][0], np.where(flipBoard == 3)[1][0])

[0 2 0 2 0 2 0 2]
[2 0 2 0 2 0 2 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 3 0]
[0 1 0 1 0 1 0 1]
[1 0 1 0 1 0 1 0]
5 6


In [20]:
data = '3;1,2;0'
selectedPiece, move = data.split(',')
selectedPiece = tuple(map(int, selectedPiece.split(';')))
move = tuple(map(int, move.split(';')))

selectedPiece, move

((3, 1), (2, 0))

In [21]:
selectedPiece = f"{selectedPiece[0]};{selectedPiece[1]}"
move = f"{move[0]};{move[1]}"
data = f"{selectedPiece},{move}"
data

'3;1,2;0'

In [22]:
import os
pickerwindowBG = np.zeros((300, 512, 3), np.uint8)
hsvColorSlider()
(hmin, hmax, smin, smax, vmin, vmax) = getHSV()
cv.imshow('HSV Color Slider', pickerwindowBG)

# hue_min = 0
# hue_max = 179
# sat_min = 0
# sat_max = 255
# val_min = 124
# val_max = 255

numGames = 0
startedFlag = False

clear = lambda: os.system('cls')

while numGames < 50:
    if startedFlag == False:
        print("Game started")
        startedFlag = True
        turn, prevBoard, bot, our = initGame(numGames)

    # print(bot, our, numGames)

    screen = ImageGrab.grab(all_screens=True)
    # print(f"Resolution: {screen.size}, Mode: {screen.mode}, Format: {screen.format}")

    # screen right : (1920, 0, 3840, 1080)
    # screen left : (0, 0, 1920, 1080)

    # screen = screen.crop((1920, 0, 3840, 1080))
    screen = np.array(screen)

    screen_binary = convert_to_binary(screen)
    ret, corners = detect_checkers_board(screen)
    screen = cv.cvtColor(screen, cv.COLOR_BGR2RGB)

    if ret:
        print("Checkers board detected")
        screen_binary = cv.cvtColor(screen_binary, cv.COLOR_GRAY2RGB)

        currBoard, _ = getBoardState(screen_binary, corners)
        # get the pieces
        pieces = detect_pieces(screen_binary, corners)
        # print(len(pieces), pieces)
        for piece in pieces:
            x, y, w, h = piece
            cv.rectangle(screen_binary, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # update board state
        # newBoard, sorted_pieces = update_board_state(screen_binary, corners)
        # if turn == bot:
        #     selected, move = getMove(prevBoard, newBoard)

        screen_binary = crop_board(screen_binary, corners)
        
        cv.imshow('screen_binary', screen_binary)

        # clear()
        # for row in currBoard:
        #     print(row)
    else:
        print("Checkers board not detected")
        # check game end
        endimg = cv.imread('end.png')
        matchinfo = match_images(screen, endimg)
        loc, w, h = matchinfo
        if loc[0].size > 0:
            print("Game ended")
            for pt in zip(*loc[::-1]):
                cv.rectangle(screen, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
            
            # reset the board
            pyautogui.moveTo(loc[1][-1] + 200, loc[0][-1] + 310)
            pyautogui.click()
            numGames += 1
            startedFlag = False

        else:
            print("Game not ended")
    # show screen
    # cv.imshow('screen', screen)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cv.destroyAllWindows()

Game started
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ended
Checkers board not detected
Game not ende

In [23]:
# get the board corners
if corners is not None:
    top_left, top_right, bottom_left, bottom_right = get_board_corners(corners)
    print(f"Top left: {top_left}, Top right: {top_right}, Bottom left: {bottom_left}, Bottom right: {bottom_right}")
    coords = get_board_squares((1, 0), corners)
    print(coords)

Top left: (1249.494, 259.57684), Top right: (1753.0778, 259.9861), Bottom left: (1249.494, 763.8835), Bottom right: (1752.5414, 764.1605)
((1280.8535766601562, 353.55682373046875), 62.719116, 62.65332)


In [24]:
if corners is not None:
    for i in range(8):
        for j in range(8):
            coords, x_delta, y_delta = get_board_squares((i, j), corners)
            print(f"Square {i}, {j}: {coords}, {x_delta}, {y_delta}")
            # pyautogui.moveTo(coords)

Square 0, 0: (1280.8535766601562, 290.90350341796875), 62.7191162109375, 62.6533203125
Square 0, 1: (1343.5726928710938, 290.90350341796875), 62.7191162109375, 62.6533203125
Square 0, 2: (1406.2918090820312, 290.90350341796875), 62.7191162109375, 62.6533203125
Square 0, 3: (1469.0109252929688, 290.90350341796875), 62.7191162109375, 62.6533203125
Square 0, 4: (1531.7300415039062, 290.90350341796875), 62.7191162109375, 62.6533203125
Square 0, 5: (1594.4491577148438, 290.90350341796875), 62.7191162109375, 62.6533203125
Square 0, 6: (1657.1682739257812, 290.90350341796875), 62.7191162109375, 62.6533203125
Square 0, 7: (1719.8873901367188, 290.90350341796875), 62.7191162109375, 62.6533203125
Square 1, 0: (1280.8535766601562, 353.55682373046875), 62.7191162109375, 62.6533203125
Square 1, 1: (1343.5726928710938, 353.55682373046875), 62.7191162109375, 62.6533203125
Square 1, 2: (1406.2918090820312, 353.55682373046875), 62.7191162109375, 62.6533203125
Square 1, 3: (1469.0109252929688, 353.55682

In [25]:
list = pieces.copy()
print(len(list))
# list = sorted(list, key=lambda x: x[0])
list = sorted(list, key=lambda x: x[0])
print(list)
print(list[10])

16
[(1261, 334, 41, 42), (1263, 715, 35, 32), (1323, 270, 42, 42), (1326, 653, 35, 31), (1386, 334, 42, 41), (1389, 717, 35, 30), (1449, 270, 42, 42), (1452, 654, 35, 32), (1512, 334, 42, 42), (1515, 716, 32, 31), (1575, 270, 42, 42), (1578, 654, 32, 32), (1638, 334, 41, 42), (1641, 716, 32, 33), (1701, 270, 41, 42), (1704, 653, 32, 31)]
(1575, 270, 42, 42)


In [26]:
pyautogui.moveTo(list[8][0], list[8][1])

In [27]:
# register the pieces to the board matrix
board = np.zeros((8, 8), dtype=int)
sorted_pieces = []
for row in range(8):
    for col in range(8):
        square = (row, col)
        coords, x_delta, y_delta = get_board_squares(square, corners)
        for idx, piece in enumerate(pieces):
            x, y, _, _ = piece
            if coords[0] - x_delta/2 < x < coords[0] + x_delta/2 and coords[1] - y_delta/2 < y < coords[1] + y_delta/2:
                board[row][col] = 1
                sorted_pieces.append(piece, square, coords)
                print(idx, piece, coords, square)
                break
                # pyautogui.moveTo((coords[0] - x_delta/2, coords[1] - y_delta/2), duration=0.25)
                # pyautogui.moveTo((coords[0] + x_delta/2, coords[1] + y_delta/2), duration=0.25)


for row in board:
    print(row)

TypeError: append() takes exactly one argument (3 given)

In [ ]:
newcoords = []
for coords in list:
    x, y, w, h = coords
    pyautogui.moveTo(x + w/2, y + h/2)
    newcoords.append((x + w/2, y + h/2))
print(newcoords)

[(784.0, 310.0), (910.0, 310.0), (1037.0, 310.0), (1163.0, 310.0), (720.5, 373.0), (973.5, 373.0), (1100.0, 373.0), (910.0, 436.5), (1099.5, 624.0), (909.5, 687.5), (783.0, 689.5), (1036.0, 689.5), (720.5, 750.5), (847.0, 751.0), (973.5, 750.5), (1100.0, 751.0)]
